In [1]:
import pandas as pd
import urllib.request

In [2]:
df = pd.read_csv('infiltration_odk.csv')
df.columns

Index(['observer', 'phone', 'date', 'site_description', 'name', 'site_photo',
       'lat', 'lon', 'altitude', 'accuracy', 'ring_1:inf_time1',
       'ring_1:inf_time2', 'ring_1:inf_time3', 'ring_1:inf_time4', 'geometry',
       'pk'],
      dtype='object')

In [3]:
from obs.models import Observation
obs= pd.DataFrame(list(Observation.objects.filter(type_id=19).values()))
obs = obs[0:29] #get only the recent entries

In [4]:
df = obs.merge(df,left_on='site_id', right_on='pk', how="inner")


In [5]:
df['filename'] = df['id'].map(str) + '.jpg'

In [7]:
df = df[['site_photo','filename']]
df = df.dropna() NaNs will wreck iteration
df = df.reset_index() #otherwise index has skips
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 28
Data columns (total 3 columns):
site_photo    27 non-null object
id            27 non-null int64
filename      27 non-null object
dtypes: int64(1), object(2)
memory usage: 864.0+ bytes


In [26]:
#urlretrieve does not accept headers which are in the url. Must use urlopen
for i in range(len(df)):
    req = urllib.request.urlopen(df.loc[i,'site_photo'])
    f = open(df.loc[i,'filename'],"wb")    # wb = write binary
    content = req.read()
    f.write(content)
    f.close()